## Approximation of reconstructed bands viewed from high-symmetry lines

In [ ]:
import warnings as wn
wn.filterwarnings("ignore")

import os
import numpy as np
import fuller
from mpes import analysis as aly
import matplotlib.pyplot as plt
import matplotlib as mpl
from tqdm import tqdm_notebook as tqdm
%matplotlib inline

In [ ]:
# mpl.rcParams['font.family'] = 'sans-serif'
# mpl.rcParams['font.sans-serif'] = 'Arial'
mpl.rcParams['axes.linewidth'] = 2
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42

# Create plot folder if needed
if not os.path.exists('../results/figures'):
    os.mkdir('../results/figures')

In [ ]:
# Generate hexagonal Zernike basis
bss = fuller.generator.ppz.hexike_basis(nterms=400, npix=175, vertical=True, outside=0)

with np.load('../data/processed/LDARecon_Approx_Polynomials.npz') as fl:
    bandcuts = fl['bands']

In [ ]:
center = np.array([88.04458508, 85.93048041])
corners = np.array([[ 43.96685519, 162.03029721],
       [132.1174288 , 162.02202934],
       [175.73285949,  85.93552008],
       [132.11926251,   9.83338561],
       [ 43.96347185,   9.85004975],
       [  0.36015304,  85.93011921]])

In [ ]:
# Define high-symmetry points
G = center
K = corners[0,:]
K1 = corners[1,:]
M = (K + K1) / 2

# Define high-symmetry lines (k-path)
pathPoints = np.asarray([G, M, K, G])
nGM, nMK, nKG = 70, 39, 79
segPoints = [nGM, nMK, nKG]
rowInds, colInds, pathInds = aly.points2path(pathPoints[:,0], pathPoints[:,1], npoints=segPoints)
nSegPoints = len(rowInds)
bandDiagramRecon = aly.bandpath_map(np.moveaxis(bandcuts, 0, 2), pathr=rowInds, pathc=colInds, eaxis=2)

lda_shift = -0.86813 + 0.15 # Global energy shift

In [ ]:
# Test plot
plt.plot(bandDiagramRecon.T)
plt.axis('off');

### Approximation of all 14 energy bands by polynomials in coefficient order

In [ ]:
# Approximation by polynomials in coefficient order
allbcf = []
allapprox, approx = {}, []
nterms = [5, 45, 150]
for iband in tqdm(range(14)):
    brec = np.nan_to_num(bandcuts[iband,...])
    bcf = fuller.generator.decomposition_hex2d(brec, nterms=400, bases=bss, ret='coeffs')
    
    allbcf.append(bcf)
    for nt in nterms:
        currcf = bcf.copy()
        currcf[nt:] = 0
        recon = fuller.generator.reconstruction_hex2d(currcf, bss)
        try:
            allapprox[str(nt)].append(recon)
        except:
            allapprox[str(nt)] = []
            allapprox[str(nt)].append(recon)

allbcf = np.asarray(allbcf)
for k, v in allapprox.items():
    allapprox[k] = np.asarray(v)

In [ ]:
bci_approx = {}
for k in allapprox.keys():
    bci_approx[k] = aly.bandpath_map(np.moveaxis(allapprox[k], 0, 2), pathr=rowInds, pathc=colInds, eaxis=2)

### Supplementary Figure 14i

In [ ]:
f, axs = plt.subplots(3, 1, figsize=(7.5, 14))

for i, nt in enumerate(nterms):
    axs[i].plot(bandDiagramRecon.T - lda_shift, c='b')
    axs[i].plot(bandDiagramRecon[-1,:] - lda_shift, c='b', label='Reconstruction')
    axs[i].plot(bci_approx[str(nt)].T - lda_shift, c='r')
    axs[i].plot(bci_approx[str(nt)][-1,:] - lda_shift, c='r', label='Approximation')
    axs[i].set_xlim([0, nSegPoints])
    axs[i].tick_params(axis='y', length=8, width=2, labelsize=15)
    axs[i].set_yticks(np.arange(-8, 0.1, 1))
    axs[i].set_ylim([-7.5, 0.8])
    axs[i].set_ylabel('Energy (eV)', fontsize=15)
    axs[i].set_title(str(nt)+' terms', x=0.76, y=0.9, fontsize=15)
    axs[i].legend(fontsize=15, frameon=False, borderpad=0, bbox_to_anchor=(0.5, 0.2))
#     axs[i].axhline(y=0, lw=2, c='k', ls='dashed')
    if i < 2:
        axs[i].tick_params(axis='x', length=0, labelsize=0)
    
    for p in pathInds[:-1]:
        axs[i].axvline(x=p, c='k', ls='--', lw=2, dashes=[4, 2])

axs[-1].set_xticks(pathInds)
axs[-1].set_xticklabels(['$\overline{\Gamma}$', '$\overline{\mathrm{M}}$',
                           '$\overline{\mathrm{K}}$', '$\overline{\Gamma}$'])
axs[-1].tick_params(axis='x', length=8, width=2, labelsize=15)
plt.suptitle('Approximation by polynomials in default order', y=0.9, fontsize=15)
plt.subplots_adjust(hspace=0.08, wspace=0)

plt.savefig('../results/figures/sfig_14i.png', dpi=300, bbox_inches='tight', transparent=True)

### Approximation of all 14 energy bands by polynomials in default order

In [ ]:
# Approximation by polynomials in default order
allbcf = []
allapproxx, approx = {}, []
nterms = [5, 45, 150]
for iband in tqdm(range(14)):
    brec = np.nan_to_num(bandcuts[iband,...])
    bcf = fuller.generator.decomposition_hex2d(brec, nterms=400, bases=bss, ret='coeffs')
    
    order = np.argsort(np.abs(bcf))[::-1]
    ordcf = bcf[order]
    ordbss = bss[order,...]
    
    allbcf.append(bcf)
    for nt in nterms:
        currcf = ordcf.copy()
        currcf[nt:] = 0
        recon = fuller.generator.reconstruction_hex2d(currcf, ordbss)
        try:
            allapproxx[str(nt)].append(recon)
        except:
            allapproxx[str(nt)] = []
            allapproxx[str(nt)].append(recon)

allbcf = np.asarray(allbcf)
for k, v in allapproxx.items():
    allapproxx[k] = np.asarray(v)

In [ ]:
bci_approxx = {}
for k in allapproxx.keys():
    bci_approxx[k] = aly.bandpath_map(np.moveaxis(allapproxx[k], 0, 2), pathr=rowInds, pathc=colInds, eaxis=2)

### Supplementary Figure 14j

In [ ]:
f, axs = plt.subplots(3, 1, figsize=(7.5, 14))

for i, nt in enumerate(nterms):
    axs[i].plot(bandDiagramRecon.T - lda_shift, c='b')
    axs[i].plot(bandDiagramRecon[-1,:] - lda_shift, c='b', label='Reconstruction')
    axs[i].plot(bci_approxx[str(nt)].T - lda_shift, c='r')
    axs[i].plot(bci_approxx[str(nt)][-1,:] - lda_shift, c='r', label='Approximation')
    axs[i].set_xlim([0, nSegPoints])
    axs[i].tick_params(axis='y', length=8, width=2, labelsize=15)
    axs[i].set_yticks(np.arange(-8, 0.1, 1))
    axs[i].set_ylim([-7.5, 0.8])
    axs[i].set_ylabel('Energy (eV)', fontsize=15)
    axs[i].set_title(str(nt)+' terms', x=0.76, y=0.9, fontsize=15)
    axs[i].legend(fontsize=15, frameon=False, borderpad=0, bbox_to_anchor=(0.5, 0.2))
#     axs[i].axhline(y=0, lw=2, c='k', ls='dashed')
    if i < 2:
        axs[i].tick_params(axis='x', length=0, labelsize=0)
    
    for p in pathInds[:-1]:
        axs[i].axvline(x=p, c='k', ls='--', lw=2, dashes=[4, 2])

axs[-1].set_xticks(pathInds)
axs[-1].set_xticklabels(['$\overline{\Gamma}$', '$\overline{\mathrm{M}}$',
                           '$\overline{\mathrm{K}}$', '$\overline{\Gamma}$'])
axs[-1].tick_params(axis='x', length=8, width=2, labelsize=15)
plt.suptitle('Approximation by polynomials in coefficient order', y=0.9, fontsize=15)
plt.subplots_adjust(hspace=0.08, wspace=0)

plt.savefig('../results/figures/sfig_14j.png', dpi=300, bbox_inches='tight', transparent=True)

In [ ]:
# np.savez('LDARecon_approx_bands.npz', allapprox=allapprox, allapproxord=allapproxx,
#          linecut=bci_approx, linecutord=bci_approxx)